# 6    | Data Aggregation and Group Operations

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### GroupBy Basics

In [2]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
# create group variable to calculate aggregate summary
grouped = df['data1'].groupby(df['key1'])
grouped.mean()

key1
a   -0.471802
b   -0.396595
Name: data1, dtype: float64

In [3]:
# groupby two columnes
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -0.298130
      two    -0.819146
b     one     0.266164
      two    -1.059353
Name: data1, dtype: float64

In [4]:
# read as df
means.unstack()

key2,one,two
key1,,
a,-0.298130,-0.819146
b,0.266164,-1.059353


In [5]:
# get 
df.groupby(['key1', 'key2']).size().unstack()

key2,one,two
key1,,
a,2,1
b,1,1


### Grouping with dictionaries

In [6]:
# return key names
for name, group in df.groupby('key1'):
    print(name)

a
b


In [7]:
# return means for each key name and dataset
for name, group in df.groupby('key1'):
    print(group)

  key1 key2     data1     data2
0    a  one -0.398813 -1.498773
1    a  two -0.819146 -1.360038
4    a  one -0.197446 -0.118984
  key1 key2     data1     data2
2    b  one  0.266164 -0.289575
3    b  two -1.059353  0.099677


In [9]:
# return df of second key in dict
pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,0.266164,-0.289575
3,b,two,-1.059353,0.099677


In [12]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

,a,b,c,d,e
Joe,0.311182,0.066078,0.001965,0.131967,-1.350212
Steve,0.057993,0.930326,-1.699144,-0.720566,0.104066
Wes,1.259205,NaN,NaN,0.627739,0.596353
Jim,-0.911582,0.831903,-2.282147,0.803533,0.971079
Travis,1.321858,-0.231019,-0.678181,0.345256,0.888056


In [13]:
# groupby new mapping using dict
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,0.133932,-0.972952
Steve,-2.419711,1.092386
Wes,0.627739,1.855558
Jim,-1.478614,0.891400
Travis,-0.332925,1.978895


### Grouping by Index Levels

In [14]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      1.006911 -0.228540 -1.655622  0.151924 -0.960352
1      0.497988 -0.031200 -1.066078 -0.075484 -0.705986
2      1.368567 -1.978481  1.153089 -0.288530  1.727722
3     -1.154637 -1.392366 -0.245312 -0.743731 -1.122130

In [15]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


In [16]:
tips = pd.read_csv('data/tips.csv')
# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [17]:
# use .agg() to show multiple metrics
grouped = tips.groupby(['day', 'smoker'])
grouped_pct = grouped['tip_pct']
grouped_pct.agg([('avg', 'mean'), ('std', np.std)])

avg       std
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [18]:
# aggregate for both total and %
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [19]:
# add additional agg column
grouped.agg({'tip' : np.max, 'size' : 'sum'})
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
             'size' : 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

In [20]:
# remove row indices
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [21]:
# sort by column
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### Quantile and Bucket Analysis

In [22]:
# define quartiles
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})
quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

0     (-1.347, 0.211]
1     (-1.347, 0.211]
2      (0.211, 1.769]
3     (-1.347, 0.211]
4     (-1.347, 0.211]
5     (-1.347, 0.211]
6     (-1.347, 0.211]
7     (-1.347, 0.211]
8    (-2.912, -1.347]
9      (0.211, 1.769]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-2.912, -1.347] < (-1.347, 0.211] < (0.211, 1.769] < (1.769, 3.327]]

In [23]:
# return statistics for group
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}
grouped = frame.data2.groupby(quartiles)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-2.912, -1.347]",-2.651147,2.830291,97.0,0.073615
"(-1.347, 0.211]",-2.845043,3.082168,485.0,-0.027823
"(0.211, 1.769]",-2.734525,2.978210,379.0,0.126628
"(1.769, 3.327]",-2.293393,3.142030,39.0,0.214212


### Filling Missing Values with Group-Specific Values

In [24]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s
s.fillna(s.mean())

0    0.735101
1    0.971216
2    0.735101
3    0.742014
4    0.735101
5    0.492073
dtype: float64

In [25]:
# create df with missing values
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index=states)
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio          1.447566
New York      0.844578
Vermont            NaN
Florida      -0.547468
Oregon        0.198042
Nevada             NaN
California    0.377941
Idaho              NaN
dtype: float64

In [26]:
# fill nan with mean
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio          1.447566
New York      0.844578
Vermont       0.581559
Florida      -0.547468
Oregon        0.198042
Nevada        0.287992
California    0.377941
Idaho         0.287992
dtype: float64

In [27]:
# or specify values
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio          1.447566
New York      0.844578
Vermont       0.500000
Florida      -0.547468
Oregon        0.198042
Nevada       -1.000000
California    0.377941
Idaho        -1.000000
dtype: float64

### Group Weighted Average and Correlation

In [28]:
# add weights based on average
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})
grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg)

category
a    0.432998
b   -0.064653
dtype: float64